In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
sys.path.append('..')

import os

In [3]:
from kst.data_sets.TextSequence import TextSequence

In [4]:
from gensim.parsing.preprocessing import (
    strip_tags,
    strip_punctuation,
    strip_multiple_whitespaces,
    remove_stopwords,
    strip_short,
    strip_numeric,
    stem_text
)

In [5]:
GENSIM_CUSTOM_TEKENIZER = [
            strip_tags,
            strip_punctuation,
            strip_multiple_whitespaces,
            strip_numeric
        ]

In [6]:
my_seq = TextSequence()
with open("/Users/lukaszmaczewski/Documents/Learning/DeepNeuralNetworks/keras/data/archive/movie_lines.txt","r") as f:
    for i in range(5):
        line = f.readline().split(" +++$+++ ")
        print(line)
        my_seq.add_element(line[4],GENSIM_CUSTOM_TEKENIZER)

['L1045', 'u0', 'm0', 'BIANCA', 'They do not!\n']
['L1044', 'u2', 'm0', 'CAMERON', 'They do to!\n']
['L985', 'u0', 'm0', 'BIANCA', 'I hope so.\n']
['L984', 'u2', 'm0', 'CAMERON', 'She okay?\n']
['L925', 'u0', 'm0', 'BIANCA', "Let's go.\n"]


In [7]:
my_seq.txt_sequence

[['they', 'do', 'not'],
 ['they', 'do', 'to'],
 ['i', 'hope', 'so'],
 ['she', 'okay'],
 ['let', 's', 'go']]

In [8]:
my_seq.token_count

defaultdict(int,
            {'<BOS>': 1,
             '<EOS>': 1,
             'they': 2,
             'do': 2,
             'not': 1,
             'to': 1,
             'i': 1,
             'hope': 1,
             'so': 1,
             'she': 1,
             'okay': 1,
             'let': 1,
             's': 1,
             'go': 1})

In [9]:
my_seq.vocabulary

dict_keys(['<BOS>', '<EOS>', 'they', 'do', 'not', 'to', 'i', 'hope', 'so', 'she', 'okay', 'let', 's', 'go'])

In [10]:
for i in my_seq.pair_sequence():
    print(i)

(['they', 'do', 'not'], ['<BOS>', 'they', 'do', 'to', '<EOS>'])
(['they', 'do', 'to'], ['<BOS>', 'i', 'hope', 'so', '<EOS>'])
(['i', 'hope', 'so'], ['<BOS>', 'she', 'okay', '<EOS>'])
(['she', 'okay'], ['<BOS>', 'let', 's', 'go', '<EOS>'])
